In [1]:
import sys
print('python', sys.version)

import numpy as np
print('numpy', np.__version__)

import pandas as pd
print('pandas', pd.__version__)

import matplotlib as mpl
print('matplotlib', mpl.__version__)

import matplotlib.pyplot as plt
plt.rc("font", family="Malgun Gothic") # 한글표시 (window)
plt.rc("axes", unicode_minus=False) # x,y축 (-)부호 표시

# 레티나 디스플레이로 폰트가 선명하게 표시되도록 합니다.
from IPython.display import set_matplotlib_formats
set_matplotlib_formats("retina")

import seaborn as sns
print('seaborn', sns.__version__)

# import folium
import json

# 결과 확인을 용이하게 하기 위한 코드
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# 모든 컬럼이 표시되도록 max_columns 의 수를 지정합니다.
pd.options.display.max_columns = 39

python 3.8.5 (default, May 27 2021, 13:30:53) 
[GCC 9.3.0]
numpy 1.19.5
pandas 1.2.5
matplotlib 3.4.2


<ipython-input-1-e03686665c0d>:19: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats("retina")


seaborn 0.11.1


In [90]:
# import pandas as pd


# patient = pd.read_csv('/data/MIMIC_III/PATIENTS.csv') #환자정보_입원환자만, DOD가 죽은 날짜 Day of DEATH #죽은 날짜는 안봐도 됨.
# cpt = pd.read_csv('../data/MIMIC_III_data/CPTEVENTS.csv')
# lab = pd.read_csv('/data/MIMIC_III/LABEVENTS.csv') #외래환자포함 #죽은 날짜 기준으로 charttime 보는 것 #abnormal이 1임
diagnoses_icd = pd.read_csv('/data/MIMIC_III/DIAGNOSES_ICD.csv') #환자있음 #dis
diagnoses = pd.read_csv('/data/MIMIC_III/D_ICD_DIAGNOSES.csv') #병이름 소개 #d_icd
process_data = pd.read_csv('../guri/data/process_data.csv')

lab = pd.read_csv('/project/data/폐렴환자lab.csv')
patient = pd.read_csv('/project/data/폐렴환자.csv')

In [3]:
lab
patient

,ROW_ID,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,FLAG
0,5144,9,150750.0,50819,2149-11-14 06:07:00,NaN
1,5145,9,150750.0,50820,2149-11-14 06:07:00,NaN
2,5146,9,150750.0,50821,2149-11-14 06:07:00,abnormal
3,5147,9,150750.0,50826,2149-11-14 06:07:00,NaN
4,5148,9,150750.0,50827,2149-11-14 06:07:00,NaN
...,...,...,...,...,...,...
9356344,27427231,96442,120151.0,51248,2115-06-29 04:45:00,NaN
9356345,27427232,96442,120151.0,51249,2115-06-29 04:45:00,NaN
9356346,27427233,96442,120151.0,51250,2115-06-29 04:45:00,NaN
9356347,27427234,96442,120151.0,51254,2115-06-29 04:45:00,abnormal


,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,237,252,M,2078-03-06 00:00:00,NaN,NaN,NaN,0
1,238,253,F,2089-11-26 00:00:00,NaN,NaN,NaN,0
2,251,268,F,2132-02-21 00:00:00,2198-02-18 00:00:00,2198-02-18 00:00:00,NaN,1
3,252,269,M,2130-09-30 00:00:00,NaN,NaN,NaN,0
4,628,665,M,2052-05-20 00:00:00,2120-02-04 00:00:00,2120-02-04 00:00:00,NaN,1
...,...,...,...,...,...,...,...,...
7802,31801,43932,M,2080-06-26 00:00:00,2153-03-01 00:00:00,NaN,2153-03-01 00:00:00,1
7803,31802,43937,M,2113-03-30 00:00:00,2200-08-05 00:00:00,2200-08-05 00:00:00,2200-08-05 00:00:00,1
7804,31807,43959,F,2028-05-13 00:00:00,NaN,NaN,NaN,0
7805,31813,43982,F,2065-04-17 00:00:00,NaN,NaN,NaN,0


In [4]:
#시간 부분 제거, 날짜만 남기기
import datetime as dt

patient['DOD'] = pd.to_datetime(patient['DOD'])
patient['DOD'] = patient['DOD'].dt.date
patient

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,237,252,M,2078-03-06 00:00:00,NaT,NaN,NaN,0
1,238,253,F,2089-11-26 00:00:00,NaT,NaN,NaN,0
2,251,268,F,2132-02-21 00:00:00,2198-02-18,2198-02-18 00:00:00,NaN,1
3,252,269,M,2130-09-30 00:00:00,NaT,NaN,NaN,0
4,628,665,M,2052-05-20 00:00:00,2120-02-04,2120-02-04 00:00:00,NaN,1
...,...,...,...,...,...,...,...,...
7802,31801,43932,M,2080-06-26 00:00:00,2153-03-01,NaN,2153-03-01 00:00:00,1
7803,31802,43937,M,2113-03-30 00:00:00,2200-08-05,2200-08-05 00:00:00,2200-08-05 00:00:00,1
7804,31807,43959,F,2028-05-13 00:00:00,NaT,NaN,NaN,0
7805,31813,43982,F,2065-04-17 00:00:00,NaT,NaN,NaN,0


In [5]:
patient[patient['SUBJECT_ID']==9]

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
40,8,9,M,2108-01-26 00:00:00,2149-11-14,2149-11-14 00:00:00,2149-11-14 00:00:00,1


In [6]:
patient_sorted = patient[['SUBJECT_ID', 'EXPIRE_FLAG']].set_index('SUBJECT_ID').sort_values(['SUBJECT_ID'])#, inplace = True)
patient_sorted

,EXPIRE_FLAG
SUBJECT_ID,
9,1
21,1
31,1
36,0
37,1
...,...
99863,0
99934,0
99937,1


In [92]:
# patient
lab

# #시간 부분 제거, 날짜만 남기기 (중복 제거 전에 원래 쓴거...)
# import datetime as dt
# lab['CHARTTIME'] = pd.to_datetime(lab['CHARTTIME'])
# lab['CHARTTIME'] = lab['CHARTTIME'].dt.date
# lab

# lab_sorted = lab[['SUBJECT_ID', 'CHARTTIME', 'ITEMID', 'FLAG']].set_index('SUBJECT_ID').sort_values(['SUBJECT_ID', 'CHARTTIME'])#, inplace = True)

lab_sorted_ = lab[['SUBJECT_ID', 'CHARTTIME', 'ITEMID', 'FLAG']].drop_duplicates()
lab_sorted_

import datetime as dt
lab_sorted_['CHARTTIME'] = pd.to_datetime(lab_sorted_['CHARTTIME'])
lab_sorted_['CHARTTIME'] = lab_sorted_['CHARTTIME'].dt.date
lab_sorted_

lab_sorted = lab_sorted_.set_index('SUBJECT_ID').sort_values(['SUBJECT_ID', 'CHARTTIME'])#, inplace = True)

# lab.drop(columns = ['ROW_ID', 'HADM_ID', "ITEMID", "VALUE"])

,ROW_ID,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,FLAG
0,5144,9,150750.0,50819,2149-11-14 06:07:00,NaN
1,5145,9,150750.0,50820,2149-11-14 06:07:00,NaN
2,5146,9,150750.0,50821,2149-11-14 06:07:00,abnormal
3,5147,9,150750.0,50826,2149-11-14 06:07:00,NaN
4,5148,9,150750.0,50827,2149-11-14 06:07:00,NaN
...,...,...,...,...,...,...
9356344,27427231,96442,120151.0,51248,2115-06-29 04:45:00,NaN
9356345,27427232,96442,120151.0,51249,2115-06-29 04:45:00,NaN
9356346,27427233,96442,120151.0,51250,2115-06-29 04:45:00,NaN
9356347,27427234,96442,120151.0,51254,2115-06-29 04:45:00,abnormal


,SUBJECT_ID,CHARTTIME,ITEMID,FLAG
0,9,2149-11-14 06:07:00,50819,NaN
1,9,2149-11-14 06:07:00,50820,NaN
2,9,2149-11-14 06:07:00,50821,abnormal
3,9,2149-11-14 06:07:00,50826,NaN
4,9,2149-11-14 06:07:00,50827,NaN
...,...,...,...,...
9356344,96442,2115-06-29 04:45:00,51248,NaN
9356345,96442,2115-06-29 04:45:00,51249,NaN
9356346,96442,2115-06-29 04:45:00,51250,NaN
9356347,96442,2115-06-29 04:45:00,51254,abnormal


,SUBJECT_ID,CHARTTIME,ITEMID,FLAG
0,9,2149-11-14,50819,NaN
1,9,2149-11-14,50820,NaN
2,9,2149-11-14,50821,abnormal
3,9,2149-11-14,50826,NaN
4,9,2149-11-14,50827,NaN
...,...,...,...,...
9356344,96442,2115-06-29,51248,NaN
9356345,96442,2115-06-29,51249,NaN
9356346,96442,2115-06-29,51250,NaN
9356347,96442,2115-06-29,51254,abnormal


In [93]:
lab_sorted_

,SUBJECT_ID,CHARTTIME,ITEMID,FLAG
0,9,2149-11-14,50819,NaN
1,9,2149-11-14,50820,NaN
2,9,2149-11-14,50821,abnormal
3,9,2149-11-14,50826,NaN
4,9,2149-11-14,50827,NaN
...,...,...,...,...
9356344,96442,2115-06-29,51248,NaN
9356345,96442,2115-06-29,51249,NaN
9356346,96442,2115-06-29,51250,NaN
9356347,96442,2115-06-29,51254,abnormal


In [94]:
lab_sorted['FLAG'].replace('abnormal', 1, inplace = True) #abnormal을 1로 바꿔주는 코드

lab_sorted[lab_sorted.index==9]['CHARTTIME'].max() #lab기록의 마지막 날짜 DOD랑 같은지 확인하려고 친 코드

datetime.date(2149, 11, 14)

In [95]:
lab_sorted[lab_sorted.index==9].tail(10)

,CHARTTIME,ITEMID,FLAG
SUBJECT_ID,,,
9,2149-11-14,51279,1
9,2149-11-14,51301,1
9,2149-11-14,50800,NaN
9,2149-11-14,50802,NaN
9,2149-11-14,50804,NaN
9,2149-11-14,50808,1
9,2149-11-14,50812,NaN
9,2149-11-14,50813,NaN
9,2149-11-14,50816,NaN


In [10]:
lab_sorted[lab_sorted.index==9]['CHARTTIME'].unique()[-10:]

array([datetime.date(2149, 11, 9), datetime.date(2149, 11, 10),
       datetime.date(2149, 11, 11), datetime.date(2149, 11, 12),
       datetime.date(2149, 11, 13), datetime.date(2149, 11, 14)],
      dtype=object)

In [11]:
lab_sorted.index.unique()

Int64Index([    9,    21,    31,    36,    37,    41,    68,    85,    94,
              101,
            ...
            99666, 99781, 99817, 99822, 99836, 99863, 99934, 99937, 99944,
            99985],
           dtype='int64', name='SUBJECT_ID', length=7799)

In [13]:
# pd.merge(left=lab_sorted, right= patient_sorted, how='inner', left_on=lab_sorted.index, right_on=patient_sorted.index)#, axis = 1)
#이 cell 안쓸거임
#https://yganalyst.github.io/data_handling/Pd_12/

In [12]:
#expire_flag는 y(label)값. 함께 보려고 lab과 patient 쓰는 것만 붙여줌.
merge_lab_pat = pd.merge(left=lab_sorted, right= patient_sorted, how='inner', left_index=True,right_index=True)

In [81]:
# merge_lab_pat.drop_duplicates(inplace = True) #순서가 여기가 아님

In [79]:
merge_lab_pat

,CHARTTIME,ITEMID,FLAG,EXPIRE_FLAG
SUBJECT_ID,,,,
9,2149-11-09,50856,NaN,1
9,2149-11-09,50868,NaN,1
9,2149-11-09,50879,NaN,1
9,2149-11-09,50880,NaN,1
9,2149-11-09,50882,1,1
...,...,...,...,...
99985,2182-03-14,50993,NaN,0
99985,2182-03-14,51000,NaN,0
99985,2182-03-14,51006,1,0


In [13]:
# value가 그냥 들어있음.
dic = {}
for i in lab_sorted.index.unique():
    dic[i] = lab_sorted[lab_sorted.index==i]['CHARTTIME'].unique()[-10:]
#     lab_sorted[lab_sorted.index==i]['CHARTTIME'].unique()

In [14]:
dic

{9: array([datetime.date(2149, 11, 9), datetime.date(2149, 11, 10),
        datetime.date(2149, 11, 11), datetime.date(2149, 11, 12),
        datetime.date(2149, 11, 13), datetime.date(2149, 11, 14)],
       dtype=object),
 21: array([datetime.date(2135, 1, 10), datetime.date(2135, 1, 11),
        datetime.date(2135, 1, 30), datetime.date(2135, 1, 31),
        datetime.date(2135, 2, 1), datetime.date(2135, 2, 2),
        datetime.date(2135, 2, 3), datetime.date(2135, 2, 4),
        datetime.date(2135, 2, 5), datetime.date(2135, 2, 6)], dtype=object),
 31: array([datetime.date(2108, 8, 22), datetime.date(2108, 8, 23),
        datetime.date(2108, 8, 24), datetime.date(2108, 8, 25),
        datetime.date(2108, 8, 26), datetime.date(2108, 8, 27),
        datetime.date(2108, 8, 28), datetime.date(2108, 8, 29),
        datetime.date(2108, 8, 30)], dtype=object),
 36: array([datetime.date(2134, 5, 11), datetime.date(2134, 5, 12),
        datetime.date(2134, 5, 13), datetime.date(2134, 5, 14),

# 10개 안뽑혀있음..

In [15]:
data = merge_lab_pat.copy()
data1 = data[:30]
data1

,CHARTTIME,ITEMID,FLAG,EXPIRE_FLAG
SUBJECT_ID,,,,
9,2149-11-09,50856,NaN,1
9,2149-11-09,50868,NaN,1
9,2149-11-09,50879,NaN,1
9,2149-11-09,50880,NaN,1
9,2149-11-09,50882,1,1
9,2149-11-09,50902,NaN,1
9,2149-11-09,50910,NaN,1
9,2149-11-09,50911,NaN,1
9,2149-11-09,50912,NaN,1


In [ ]:
# 이하는 오늘의 수확을 위한 실행

# 공df

In [50]:
mul_col = ['CHARTTIME', 'ITEMID', 'FLAG', 'EXPIRE_FLAG']*10

In [65]:
list(range(10,0,-1))
[range(10,0,-1)]

[10, 9, 8, 7, 6, 5, 4, 3, 2, 1]

[range(10, 0, -1)]

In [72]:
mul_ = []
for i in list(range(10,0,-1)):
    mul_.extend([i,i,i,i])

In [73]:
mul_

[10,
 10,
 10,
 10,
 9,
 9,
 9,
 9,
 8,
 8,
 8,
 8,
 7,
 7,
 7,
 7,
 6,
 6,
 6,
 6,
 5,
 5,
 5,
 5,
 4,
 4,
 4,
 4,
 3,
 3,
 3,
 3,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 1]

In [74]:
d = pd.DataFrame(columns = [mul_,mul_col])
d.index.name = 'SUBJECT_ID'

In [75]:
d

Empty DataFrame
Columns: [(10, CHARTTIME), (10, ITEMID), (10, FLAG), (10, EXPIRE_FLAG), (9, CHARTTIME), (9, ITEMID), (9, FLAG), (9, EXPIRE_FLAG), (8, CHARTTIME), (8, ITEMID), (8, FLAG), (8, EXPIRE_FLAG), (7, CHARTTIME), (7, ITEMID), (7, FLAG), (7, EXPIRE_FLAG), (6, CHARTTIME), (6, ITEMID), (6, FLAG), (6, EXPIRE_FLAG), (5, CHARTTIME), (5, ITEMID), (5, FLAG), (5, EXPIRE_FLAG), (4, CHARTTIME), (4, ITEMID), (4, FLAG), (4, EXPIRE_FLAG), (3, CHARTTIME), (3, ITEMID), (3, FLAG), (3, EXPIRE_FLAG), (2, CHARTTIME), (2, ITEMID), (2, FLAG), (2, EXPIRE_FLAG), (1, CHARTTIME), (1, ITEMID), (1, FLAG), (1, EXPIRE_FLAG)]
Index: []

[0 rows x 40 columns]

# fillna(0), 실행해줘야 하는 cell

In [19]:
data.fillna(0, inplace = True)

In [20]:
data

,CHARTTIME,ITEMID,FLAG,EXPIRE_FLAG
SUBJECT_ID,,,,
9,2149-11-09,50856,0,1
9,2149-11-09,50868,0,1
9,2149-11-09,50879,0,1
9,2149-11-09,50880,0,1
9,2149-11-09,50882,1,1
...,...,...,...,...
99985,2182-03-14,51006,1,0
99985,2182-03-14,51146,0,0
99985,2182-03-14,51200,0,0


In [21]:
# 오늘의 수확
d = pd.DataFrame(columns = ['CHARTTIME', 'ITEMID', 'FLAG', 'EXPIRE_FLAG'])
d.index.name = 'SUBJECT_ID'
for i,j in enumerate(data.index.unique()): #i는 0부터, j는 subject_id
    if data.loc[j].index.unique()[0] == j: #index가 j일 때
        dj = data.loc[j]
#         dic[j]
#         (data[data.index==j])['CHARTTIME'].unique()[-10:] #dic[j]랑 같음
#         s = data.loc[j].index.unique()[0] # 이 아이디어로 charttime 갖고와.
        s = data.loc[j]['CHARTTIME'].unique()[-10:]
#         s
#         s[0]
        for k,l in enumerate(s):#k는 0부터, l는 s(charttime)
#             l
            if data.loc[j]['CHARTTIME'].unique()[-10:][k] == s[k]:
                m = s[k]
                dj_ = dj[dj['CHARTTIME']==m]
#                 data.loc[j][data.loc[j]['CHARTTIME']==m]
#                 if data.loc[j][data.loc[j]['CHARTTIME']==m]:
                dj_.columns = pd.MultiIndex.from_product([[m], dj_.columns])
#                 dj_.drop(columns = ['CHARTTIME'], axis=1) #charttime drop 어떡함?
                d = pd.merge(left=d, right= dj_, how='outer', left_index=True,right_index=True) #한 줄 머지가 끝나면 다음줄로 넘어가야지
                # 같은 SUBJEECT_ID에 대해서 계산한 결과겠지?
                d
        
    if 
#                 break
#             break
#         break
#     break
#                 d.append(df_)
                
#                 data.loc[j][data.loc[j]['CHARTTIME']==m].columns = pd.MultiIndex.from_product([[m], data.loc[j][data.loc[j]['CHARTTIME']==m].columns])
#                 data[data.index==j].columns = pd.MultiIndex.from_product([[m], data[data.index==j].columns])
#                 data

/usr/local/lib/python3.8/dist-packages/pandas/core/reshape/merge.py:648: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


KeyboardInterrupt: 

In [22]:
d

,CHARTTIME,ITEMID,FLAG,EXPIRE_FLAG,"(2149-11-09 00:00:00, CHARTTIME)","(2149-11-09 00:00:00, ITEMID)","(2149-11-09 00:00:00, FLAG)","(2149-11-09 00:00:00, EXPIRE_FLAG)","(2135-01-10 00:00:00, CHARTTIME)","(2135-01-10 00:00:00, ITEMID)","(2135-01-10 00:00:00, FLAG)","(2135-01-10 00:00:00, EXPIRE_FLAG)","(2108-08-22 00:00:00, CHARTTIME)","(2108-08-22 00:00:00, ITEMID)","(2108-08-22 00:00:00, FLAG)","(2108-08-22 00:00:00, EXPIRE_FLAG)","(2134-05-11 00:00:00, CHARTTIME)","(2134-05-11 00:00:00, ITEMID)","(2134-05-11 00:00:00, FLAG)",...,"(2156-09-23 00:00:00, ITEMID)","(2156-09-23 00:00:00, FLAG)","(2156-09-23 00:00:00, EXPIRE_FLAG)","(2152-10-02 00:00:00, CHARTTIME)","(2152-10-02 00:00:00, ITEMID)","(2152-10-02 00:00:00, FLAG)","(2152-10-02 00:00:00, EXPIRE_FLAG)","(2143-08-26 00:00:00, CHARTTIME)","(2143-08-26 00:00:00, ITEMID)","(2143-08-26 00:00:00, FLAG)","(2143-08-26 00:00:00, EXPIRE_FLAG)","(2143-01-17 00:00:00, CHARTTIME)","(2143-01-17 00:00:00, ITEMID)","(2143-01-17 00:00:00, FLAG)","(2143-01-17 00:00:00, EXPIRE_FLAG)","(2161-01-01 00:00:00, CHARTTIME)","(2161-01-01 00:00:00, ITEMID)","(2161-01-01 00:00:00, FLAG)","(2161-01-01 00:00:00, EXPIRE_FLAG)"
SUBJECT_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9,NaN,NaN,NaN,NaN,2149-11-09,50856.0,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,2149-11-09,50868.0,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,2149-11-09,50879.0,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,2149-11-09,50880.0,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,2149-11-09,50882.0,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7234,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2161-01-01,50816.0,0,1.0
7234,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2161-01-01,50817.0,0,1.0
7234,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2161-01-01,50819.0,0,1.0


In [40]:
# pd.DataFrame(dj_['2149-11-09','CHARTTIME'])
dj_.xs('2149-11-09', level = 0, axis = 1)["CHARTTIME"]#같은 결과임
# dj_.drop([dj_.xs('2149-11-09', level = 0, axis = 1)["CHARTTIME"]], axis = 1)

SUBJECT_ID
9    2149-11-09
9    2149-11-09
9    2149-11-09
9    2149-11-09
9    2149-11-09
        ...    
9    2149-11-09
9    2149-11-09
9    2149-11-09
9    2149-11-09
9    2149-11-09
Name: CHARTTIME, Length: 94, dtype: object

TypeError: 'Series' objects are mutable, thus they cannot be hashed

In [47]:
data.loc[j][data.loc[j]['CHARTTIME']==m]

,CHARTTIME,ITEMID,FLAG,EXPIRE_FLAG
SUBJECT_ID,,,,
99985,2182-03-14,51244,NaN,0
99985,2182-03-14,51248,NaN,0
99985,2182-03-14,51249,NaN,0
99985,2182-03-14,51250,NaN,0
99985,2182-03-14,51254,NaN,0
99985,2182-03-14,51256,NaN,0
99985,2182-03-14,51265,NaN,0
99985,2182-03-14,51277,NaN,0
99985,2182-03-14,51279,1,0


In [33]:
data.loc[j].index.unique()[0]

99985

In [34]:
data.loc[j][data.loc[j]['CHARTTIME'] == m]

,CHARTTIME,ITEMID,FLAG,EXPIRE_FLAG
SUBJECT_ID,,,,
99985,2182-03-14,51244,NaN,0
99985,2182-03-14,51248,NaN,0
99985,2182-03-14,51249,NaN,0
99985,2182-03-14,51250,NaN,0
99985,2182-03-14,51254,NaN,0
99985,2182-03-14,51256,NaN,0
99985,2182-03-14,51265,NaN,0
99985,2182-03-14,51277,NaN,0
99985,2182-03-14,51279,1,0


In [148]:
merge_lab_pat.loc[9].index.unique()

Int64Index([9], dtype='int64', name='SUBJECT_ID')

In [144]:
merge_lab_pat.index.unique()#.loc[9]

Int64Index([    9,    21,    31,    36,    37,    41,    68,    85,    94,
              101,
            ...
            99666, 99781, 99817, 99822, 99836, 99863, 99934, 99937, 99944,
            99985],
           dtype='int64', name='SUBJECT_ID', length=7799)

In [ ]:
data_cp = data.copy()
data_cp.reset_index(inplace = True)
data_cp
data_gb = data_cp.groupby(['SUBJECT_ID', 'ITEMID']).EXPIRE_FLAG.mean()
pd.DataFrame(data_gb)

In [ ]:
# 이하는 multi index말고는 뭔생각인지 모르겠음. 쓸모 없음.

In [21]:
data = process_data.copy()
data
data_set = data.reset_index().set_index('환자id')
data_set
# data_drop =
data_set.drop(columns=['index'], axis=1)
data_set
# data_drop

,환자id,ITEM_50800,ITEM_50801,ITEM_50802,ITEM_50803,ITEM_50804,ITEM_50805,ITEM_50806,ITEM_50808,ITEM_50809,ITEM_50810,ITEM_50811,ITEM_50812,ITEM_50813,ITEM_50814,ITEM_50815,ITEM_50816,ITEM_50817,ITEM_50818,...,ITEM_51510,ITEM_51511,ITEM_51512,ITEM_51513,ITEM_51514,ITEM_51515,ITEM_51516,ITEM_51517,ITEM_51518,ITEM_51519,ITEM_51520,ITEM_51521,ITEM_51523,ITEM_51525,ITEM_51526,ITEM_51529,ITEM_51533,ITEM_51537,EXPIRE_FLAG
0,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,21,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,31,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,36,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,37,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7794,99863,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
7795,99934,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
7796,99937,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
7797,99944,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


,index,ITEM_50800,ITEM_50801,ITEM_50802,ITEM_50803,ITEM_50804,ITEM_50805,ITEM_50806,ITEM_50808,ITEM_50809,ITEM_50810,ITEM_50811,ITEM_50812,ITEM_50813,ITEM_50814,ITEM_50815,ITEM_50816,ITEM_50817,ITEM_50818,...,ITEM_51510,ITEM_51511,ITEM_51512,ITEM_51513,ITEM_51514,ITEM_51515,ITEM_51516,ITEM_51517,ITEM_51518,ITEM_51519,ITEM_51520,ITEM_51521,ITEM_51523,ITEM_51525,ITEM_51526,ITEM_51529,ITEM_51533,ITEM_51537,EXPIRE_FLAG
환자id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
21,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
31,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
36,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
37,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99863,7794,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
99934,7795,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
99937,7796,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


,ITEM_50800,ITEM_50801,ITEM_50802,ITEM_50803,ITEM_50804,ITEM_50805,ITEM_50806,ITEM_50808,ITEM_50809,ITEM_50810,ITEM_50811,ITEM_50812,ITEM_50813,ITEM_50814,ITEM_50815,ITEM_50816,ITEM_50817,ITEM_50818,ITEM_50819,...,ITEM_51510,ITEM_51511,ITEM_51512,ITEM_51513,ITEM_51514,ITEM_51515,ITEM_51516,ITEM_51517,ITEM_51518,ITEM_51519,ITEM_51520,ITEM_51521,ITEM_51523,ITEM_51525,ITEM_51526,ITEM_51529,ITEM_51533,ITEM_51537,EXPIRE_FLAG
환자id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
21,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
31,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
36,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
37,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99863,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
99934,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
99937,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


,index,ITEM_50800,ITEM_50801,ITEM_50802,ITEM_50803,ITEM_50804,ITEM_50805,ITEM_50806,ITEM_50808,ITEM_50809,ITEM_50810,ITEM_50811,ITEM_50812,ITEM_50813,ITEM_50814,ITEM_50815,ITEM_50816,ITEM_50817,ITEM_50818,...,ITEM_51510,ITEM_51511,ITEM_51512,ITEM_51513,ITEM_51514,ITEM_51515,ITEM_51516,ITEM_51517,ITEM_51518,ITEM_51519,ITEM_51520,ITEM_51521,ITEM_51523,ITEM_51525,ITEM_51526,ITEM_51529,ITEM_51533,ITEM_51537,EXPIRE_FLAG
환자id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
21,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
31,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
36,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
37,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99863,7794,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
99934,7795,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
99937,7796,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [49]:
data1 = data_drop[:10]
data1

,ITEM_50800,ITEM_50801,ITEM_50802,ITEM_50803,ITEM_50804,ITEM_50805,ITEM_50806,ITEM_50808,ITEM_50809,ITEM_50810,ITEM_50811,ITEM_50812,ITEM_50813,ITEM_50814,ITEM_50815,ITEM_50816,ITEM_50817,ITEM_50818,ITEM_50819,...,ITEM_51510,ITEM_51511,ITEM_51512,ITEM_51513,ITEM_51514,ITEM_51515,ITEM_51516,ITEM_51517,ITEM_51518,ITEM_51519,ITEM_51520,ITEM_51521,ITEM_51523,ITEM_51525,ITEM_51526,ITEM_51529,ITEM_51533,ITEM_51537,EXPIRE_FLAG
환자id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
21,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
31,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
36,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
37,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
41,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
68,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
85,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
94,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [51]:
data1[:1]

,ITEM_50800,ITEM_50801,ITEM_50802,ITEM_50803,ITEM_50804,ITEM_50805,ITEM_50806,ITEM_50808,ITEM_50809,ITEM_50810,ITEM_50811,ITEM_50812,ITEM_50813,ITEM_50814,ITEM_50815,ITEM_50816,ITEM_50817,ITEM_50818,ITEM_50819,...,ITEM_51510,ITEM_51511,ITEM_51512,ITEM_51513,ITEM_51514,ITEM_51515,ITEM_51516,ITEM_51517,ITEM_51518,ITEM_51519,ITEM_51520,ITEM_51521,ITEM_51523,ITEM_51525,ITEM_51526,ITEM_51529,ITEM_51533,ITEM_51537,EXPIRE_FLAG
환자id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [56]:
df = data1.stack(level=0, dropna=True)
pd.DataFrame(df)
# df.reset_index()#.set_index('환자id')
# 출처: https://data-make.tistory.com/126 [Data Makes Our Future]

0
환자id                 
9    ITEM_50800   0.0
     ITEM_50801   0.0
     ITEM_50802   0.0
     ITEM_50803   0.0
     ITEM_50804   0.0
...               ...
101  ITEM_51526   0.0
     ITEM_51529   0.0
     ITEM_51533   0.0
     ITEM_51537   0.0
     EXPIRE_FLAG  1.0

[6910 rows x 1 columns]

In [58]:
df.shape

(6910,)

In [ ]:
# df1 = DataFrame({'value':[1,2,3,4,5,6], 'ind1':['a','a','b','b','c','c'], 'ind2':[1,2,1,2,1,2]})

# df1 = df1.set_index(['ind1','ind2'])  # 리스트 형식으로 인덱스에 동시 전달 가능

# # df1.index = [df1['ind1'], df1['ind2']]
# # df1.drop(['ind1','ind2'],axis=1) 

# # 출처: https://data-make.tistory.com/126 [Data Makes Our Future]